On va essayer de faire marcher l'api spotify.


In [94]:
import spotipy
import pandas as pd
import requests
from spotipy.oauth2 import SpotifyClientCredentials

In [105]:
#
SPOTIFY_CLIENT_ID="4ec88fe1ae464b4a9344a2700c06e93a"   
SPOTIFY_SECRET_CLIENT_ID="0b674d20c05447e0a2098b4de70789d3"

API_KEY = "b62fc256552a9dfdae35aa99a527cd70"
BASE_URL = "http://ws.audioscrobbler.com/2.0/"

In [106]:
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=SPOTIFY_CLIENT_ID,
                                                           client_secret=SPOTIFY_SECRET_CLIENT_ID))

In [114]:
# Chargement playlists csv
df_playlists = pd.read_csv("spotify.csv")
df_playlists.head()

,mood,url
0,sad,https://open.spotify.com/playlist/37i9dQZF1DX0...
1,sad,https://open.spotify.com/playlist/6nxPNnmSE0d5...
2,sad,https://open.spotify.com/playlist/3c0Nv5CY6TIa...
3,happy,https://open.spotify.com/playlist/37i9dQZF1DXd...
4,happy,https://open.spotify.com/playlist/37i9dQZF1DWZ...


In [115]:
# DataFrame avec toutes les données
df_songs = pd.DataFrame(columns=["emotion", "song_id", "song_name", "artist_name", "album_name","genre", "release_date", "danceability", "energy", "key", "loudness", "mode", "speechiness", "acousticness", "instrumentalness", "liveness", "valence", "tempo", "duration_ms", "time_signature"])

In [116]:
for index, row in df_playlists.iterrows():
    print(index)
    emotion = row["mood"]
    playlist_id = row["url"][34:56]

    tracks = sp.playlist_tracks(playlist_id)["items"]
    for track in tracks:
        song_id = track["track"]["id"]
        song_name = track["track"]["name"]
        artist_name = track["track"]["artists"][0]["name"]
        album_name = track["track"]["album"]["name"]
        artist_genre = sp.artist(track['track']['artists'][0]['id'])
        endpoint = BASE_URL + "?method=track.getInfo" + "&api_key=" + API_KEY + "&artist=" + artist_name + "&track=" + song_name + "&format=json"
        response = requests.get(endpoint)
        song_tags = response.json()
        if len(song_tags.keys()) != 1 :
            genre = []
        else: 
            song_tags = song_tags['track']['toptags']['tag']
            list_genre = [i['name'] for i in song_tags]
            genre = list(set(artist_genre['genres'])&set(list_genre))
        release_date = track["track"]["album"]["release_date"]

        track_features = sp.audio_features(song_id)[0]

        df_songs.loc[len(df_songs)] = [emotion, 
                                       song_id, 
                                       song_name, 
                                       artist_name, 
                                       album_name,
                                       genre, 
                                       release_date, 
                                       track_features["danceability"], 
                                       track_features["energy"], 
                                       track_features["key"], 
                                       track_features["loudness"], 
                                       track_features["mode"], 
                                       track_features["speechiness"], 
                                       track_features["acousticness"], 
                                       track_features["instrumentalness"], 
                                       track_features["liveness"], 
                                       track_features["valence"], 
                                       track_features["tempo"], 
                                       track_features["duration_ms"], 
                                       track_features["time_signature"]]

# décommenter pr enregistrer en csv, c'était pr tester
df_songs.to_csv("informations_chansons.csv", index=False)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34


In [128]:
df_songs.head()

,emotion,song_id,song_name,artist_name,album_name,genre,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,sad,0qFZubf0FS0xxropyslwYJ,Sally,gigi,Sally,[],2023-03-10,0.569,0.320,1,-9.902,1,0.0386,0.520,0.000000,0.4110,0.2690,122.851,273936,3
1,sad,3MlEryrxCKZkcfX18ZLX96,Candy Necklace (feat. Jon Batiste),Lana Del Rey,Did you know that there's a tunnel under Ocean...,"[art pop, pop]",2023-03-24,0.343,0.142,11,-16.432,0,0.0395,0.871,0.010300,0.0915,0.0799,116.295,314520,4
2,sad,7dISpxgINJOQSuXUTEfB9x,Secret,Louane,Sentiments,[],2022-12-09,0.643,0.276,5,-8.481,0,0.0596,0.901,0.000000,0.1140,0.2130,149.503,191173,4
3,sad,1CS7Sd1u5tWkstBhpssyjP,Take Me to Church,Hozier,Hozier (Expanded Edition),[],2014-09-19,0.566,0.664,4,-5.303,0,0.0464,0.634,0.000000,0.1160,0.4370,128.945,241693,4
4,sad,2nC3QhMI9reBIOWutbU3Tj,Moral of the Story,Ashe,Moral of the Story,[],2019-02-14,0.572,0.406,10,-8.624,1,0.0427,0.587,0.000004,0.1020,0.2650,119.812,201084,4


In [130]:
df_songs['genre'].value_counts()

[]                              2077
[pop]                            692
[rock]                            38
[rap]                             20
[hip hop, rap]                    20
                                ... 
[trap, rap]                        1
[tropical house, electropop]       1
[indie pop, indie rock]            1
[ambient]                          1
[latin pop]                        1
Name: genre, Length: 76, dtype: int64

-------------------------------------------------
A partir de la c'est des tests
-------------------------------------------------

In [36]:
req = sp.track("2Bo0hh0yoQReC4reJav5DT")
req2 = sp.artist(req['artists'][0]['id'])
req2

{'external_urls': {'spotify': 'https://open.spotify.com/artist/5WUlDfRSoLAfcVSX1WnrxN'},
 'followers': {'href': None, 'total': 25914388},
 'genres': ['australian dance', 'australian pop', 'dance pop', 'pop'],
 'href': 'https://api.spotify.com/v1/artists/5WUlDfRSoLAfcVSX1WnrxN',
 'id': '5WUlDfRSoLAfcVSX1WnrxN',
 'images': [{'height': 640,
   'url': 'https://i.scdn.co/image/ab6761610000e5ebec40474426f4401a4203dc9f',
   'width': 640},
  {'height': 320,
   'url': 'https://i.scdn.co/image/ab67616100005174ec40474426f4401a4203dc9f',
   'width': 320},
  {'height': 160,
   'url': 'https://i.scdn.co/image/ab6761610000f178ec40474426f4401a4203dc9f',
   'width': 160}],
 'name': 'Sia',
 'popularity': 86,
 'type': 'artist',
 'uri': 'spotify:artist:5WUlDfRSoLAfcVSX1WnrxN'}

In [35]:
infos = pd.read_csv('informations_chansons.csv')
print(len(infos))
(infos['album_genre'] == '[]').sum()

3685


3685

In [88]:
# Titre et artiste de la chanson à rechercher
titre = "Bohemian Rhapsody"
artiste = "Queen"

# Endpoint pour récupérer les informations sur une chanson
endpoint = BASE_URL + "?method=track.getInfo" + "&api_key=" + API_KEY + "&artist=" + artiste + "&track=" + titre + "&format=json"

In [89]:
response = requests.get(endpoint)

In [90]:
response.json().keys()

dict_keys(['track'])

In [72]:
response.json().keys()

dict_keys(['error', 'message', 'links'])

In [42]:
# Envoi de la requête à l'API


# Analyse de la réponse JSON
data = response
genres = data["track"]["toptags"]["tag"]

# Affichage des genres associés à la chanson
print("Genres de la chanson", titre, "par", artiste, ":")
for genre in genres:
    print("- ", genre["name"])

Genres de la chanson Bohemian Rhapsody par Queen :
-  classic rock
-  rock
-  Queen
-  70s
-  80s
